In [1]:

import numpy as np
import pandas as pd
import glob
#from tqdm.auto import tqdm
import os
from datetime import datetime, timedelta, date
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
#import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
#from dash import Dash, html, dcc
import tkinter
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF8')

import warnings
warnings.filterwarnings("ignore")


In [2]:
import configparser

config = configparser.ConfigParser()
config.read('strategy_config.ini')

strategy_path = config['base']['path']
file_type = config['base']['file_type']
strategy_report_paths = glob.glob(strategy_path + '*' + file_type)

In [40]:
strategy_report_path = strategy_report_paths[-2]

In [43]:
stat_df = pd.read_excel(strategy_report_path, sheet_name='整體統計')
stat_df = stat_df.iloc[:35, :3]
stat_df.columns = ['statistics', 'value', 'ratio']

if stat_df['statistics'].iloc[4] != '交易腳本':
    raise ValueError(strategy_name +  ' 非交易腳本 ! ')
    
trading_df = pd.read_excel(strategy_report_path, sheet_name='交易分析')
trading_df['進場時間'] = pd.to_datetime(trading_df['進場時間'])
trading_df['出場時間'] = pd.to_datetime(trading_df['出場時間'].replace('--', ''))
trading_df['獲利金額'] = trading_df['獲利金額'].replace('[\$,]', '', regex=True).astype(float)
trading_df['買進金額'] = trading_df['進場價格'] * trading_df['交易數量'] * 1000
trading_df = trading_df.sort_values(by='進場時間')


In [10]:
daily_report_df = pd.read_excel( strategy_report_path, sheet_name='每日報表')
daily_report_df['Date'] = pd.to_datetime(daily_report_df['日期']) 
daily_report_df['獲利'] = daily_report_df['獲利'].replace('[\$,]', '', regex=True).astype(float)


In [7]:
backtesting_trading_days = stat_df[stat_df['statistics'] == '回測K線根數']['value'].values[0]
backtesting_date = stat_df[stat_df['statistics'] == '回測資料範圍']['value'].values[0]
#backtesting_date = backtesting_date[:23]

In [21]:
pd.to_datetime(backtesting_date[23-10:23]) - pd.to_datetime(backtesting_date[:10])

Timedelta('1826 days 00:00:00')

In [5]:

def get_daily_df(trading_df):

    start_date = trading_df['進場時間'].min() 
    end_date = trading_df['進場時間'].max() 
    start_date = date(start_date.year, start_date.month, start_date.day)
    end_date = date(end_date.year, end_date.month, end_date.day)

    cur_date = start_date 
    money_list = []

    while cur_date <= end_date:
        
        cur_trading_df = trading_df[(trading_df['進場時間'].dt.date <= cur_date) & (trading_df['出場時間'].dt.date >= cur_date)]
        money_list.append([cur_date, trading_df[trading_df['出場時間'].dt.date == cur_date]['獲利金額'].sum(), cur_trading_df['買進金額'].sum()])
        cur_date += timedelta(days=1)

    daily_money_df = pd.DataFrame(money_list)
    daily_money_df.columns = ['date', 'profit', 'holding_money']

    return daily_money_df
    

In [6]:
daily_df = get_daily_df(trading_df)

In [115]:
stat_df

,statistics,value,ratio
0,NaN,NaN,NaN
1,回測資料範圍,2017/11/14 - 2022/11/14 (還原日資料) (逐筆洗價),NaN
2,執行商品個數,965(成功)2(失敗),NaN
3,回測執行時間,2022/11/15 16:32 (費時1分44秒),NaN
4,交易腳本,選股轉交易_v6,NaN
5,初始資金,1000萬元,NaN
6,交易費用,股票0.2%(成交金額)、期貨100元(口),NaN
7,NaN,NaN,NaN
8,項目,全部,NaN
9,淨利,"1,242,728",12.43%
